In [2]:
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
from branca.colormap import linear
import ipywidgets as widgets
from IPython.display import display, HTML as ipyHTML
from IPython.display import display, IFrame

In [3]:
data = pd.read_excel('Datasets/dados_tipoescola.xlsx')
data.rename(columns={'SG_UF': 'SIGLA_UF'}, inplace=True)
print(data)

           NO_REGIAO             NO_UF SIGLA_UF NO_MUNICIPIO  \
0              Norte          Rondônia       RO  Porto Velho   
1              Norte          Rondônia       RO  Porto Velho   
2              Norte          Rondônia       RO  Porto Velho   
3              Norte          Rondônia       RO  Porto Velho   
4              Norte          Rondônia       RO  Porto Velho   
...              ...               ...      ...          ...   
217620  Centro-Oeste  Distrito Federal       DF     Brasília   
217621  Centro-Oeste  Distrito Federal       DF     Brasília   
217622  Centro-Oeste  Distrito Federal       DF     Brasília   
217623  Centro-Oeste  Distrito Federal       DF     Brasília   
217624  Centro-Oeste  Distrito Federal       DF     Brasília   

                                            NO_ENTIDADE  TP_DEPENDENCIA  
0                    EEEE ABNAEL MACHADO DE LIMA - CENE               2  
1                              EMEIEF PEQUENOS TALENTOS               3  
2        

In [4]:
aux = data['SIGLA_UF'].value_counts()
df_final = aux.reset_index()
df_final.columns = ['SIGLA_UF', 'QT_ESCOLAS']
print(df_final)

   SIGLA_UF  QT_ESCOLAS
0        SP       34099
1        MG       24293
2        BA       19611
3        MA       13635
4        RJ       13229
5        PA       12092
6        RS       11287
7        CE       10648
8        PE       10219
9        PR        9999
10       SC        7376
11       AM        6148
12       PI        5849
13       PB        5479
14       GO        5047
15       RN        4867
16       ES        4037
17       AL        3473
18       MT        3154
19       SE        2567
20       TO        2293
21       MS        1939
22       AC        1658
23       RO        1416
24       DF        1319
25       AP         950
26       RR         941


In [5]:
federal = data[data['TP_DEPENDENCIA'] == 1]
federal_uf = federal.groupby('SIGLA_UF').size().reset_index(name='QT_FED')
federal_uf = federal_uf.sort_values(by='SIGLA_UF')
print(federal_uf)

   SIGLA_UF  QT_FED
0        AC       7
1        AL      17
2        AM      18
3        AP       6
4        BA      37
5        CE      35
6        DF      11
7        ES      23
8        GO      27
9        MA      31
10       MG      82
11       MS      11
12       MT      19
13       PA      24
14       PB      26
15       PE      26
16       PI      23
17       PR      30
18       RJ      68
19       RN      30
20       RO      10
21       RR       7
22       RS      50
23       SC      39
24       SE      10
25       SP      47
26       TO      11


In [6]:
estadual = data[data['TP_DEPENDENCIA'] == 2]
estadual_uf = estadual.groupby('SIGLA_UF').size().reset_index(name='QT_EST')
estadual_uf = estadual_uf.sort_values(by='SIGLA_UF')
print(estadual_uf)

   SIGLA_UF  QT_EST
0        AC     673
1        AL     326
2        AM     789
3        AP     402
4        BA    1137
5        CE     776
6        DF     703
7        ES     588
8        GO    1015
9        MA    1313
10       MG    4294
11       MS     365
12       MT     825
13       PA    1103
14       PB     673
15       PE    1085
16       PI     762
17       PR    2140
18       RJ    1593
19       RN     820
20       RO     416
21       RR     404
22       RS    2516
23       SC    1378
24       SE     335
25       SP    6524
26       TO     591


In [7]:
municipal = data[data['TP_DEPENDENCIA'] == 3]
municipal_uf = municipal.groupby('SIGLA_UF').size().reset_index(name='QT_MUN')
municipal_uf = municipal_uf.sort_values(by='SIGLA_UF')
print(municipal_uf)

   SIGLA_UF  QT_MUN
0        AC     931
1        AL    2553
2        AM    4941
3        AP     433
4        BA   15175
5        CE    7086
6        ES    2758
7        GO    2700
8        MA   10902
9        MG   13441
10       MS    1012
11       MT    1792
12       PA    9970
13       PB    3777
14       PE    5852
15       PI    4554
16       PR    5459
17       RJ    6286
18       RN    3037
19       RO     789
20       RR     479
21       RS    5650
22       SC    4283
23       SE    1568
24       SP   13853
25       TO    1428


In [8]:
privada = data[data['TP_DEPENDENCIA'] == 4]
privada_uf = privada.groupby('SIGLA_UF').size().reset_index(name='QT_PRI')
privada_uf = privada_uf.sort_values(by='SIGLA_UF')
print(privada_uf)

   SIGLA_UF  QT_PRI
0        AC      47
1        AL     577
2        AM     400
3        AP     109
4        BA    3262
5        CE    2751
6        DF     605
7        ES     668
8        GO    1305
9        MA    1389
10       MG    6476
11       MS     551
12       MT     518
13       PA     995
14       PB    1003
15       PE    3256
16       PI     510
17       PR    2370
18       RJ    5282
19       RN     980
20       RO     201
21       RR      51
22       RS    3071
23       SC    1676
24       SE     654
25       SP   13675
26       TO     263


In [9]:
dfprincipal = df_final
for df in [federal_uf, municipal_uf, estadual_uf, privada_uf]:
    dfprincipal =dfprincipal.merge(df, on='SIGLA_UF')

print(dfprincipal)

   SIGLA_UF  QT_ESCOLAS  QT_FED  QT_MUN  QT_EST  QT_PRI
0        SP       34099      47   13853    6524   13675
1        MG       24293      82   13441    4294    6476
2        BA       19611      37   15175    1137    3262
3        MA       13635      31   10902    1313    1389
4        RJ       13229      68    6286    1593    5282
5        PA       12092      24    9970    1103     995
6        RS       11287      50    5650    2516    3071
7        CE       10648      35    7086     776    2751
8        PE       10219      26    5852    1085    3256
9        PR        9999      30    5459    2140    2370
10       SC        7376      39    4283    1378    1676
11       AM        6148      18    4941     789     400
12       PI        5849      23    4554     762     510
13       PB        5479      26    3777     673    1003
14       GO        5047      27    2700    1015    1305
15       RN        4867      30    3037     820     980
16       ES        4037      23    2758     588 

In [10]:
#adicionando manualmente a linha do DF
nova_linha = {'SIGLA_UF': 'DF', 'QT_ESCOLAS': 1319, 'QT_FED': 11, 'QT_MUN': 0, 'QT_EST':703, 'QT_PRI':605}
dfprincipal.loc[len(dfprincipal)] = nova_linha

In [11]:
dfprincipal['ESCOLA_FED'] = dfprincipal['QT_FED'] / dfprincipal['QT_ESCOLAS']*100
dfprincipal['ESCOLA_EST'] = dfprincipal['QT_EST'] / dfprincipal['QT_ESCOLAS']*100
dfprincipal['ESCOLA_MUN'] = dfprincipal['QT_MUN'] / dfprincipal['QT_ESCOLAS']*100
dfprincipal['ESCOLA_PRI'] = dfprincipal['QT_PRI'] / dfprincipal['QT_ESCOLAS']*100
print(dfprincipal)

   SIGLA_UF  QT_ESCOLAS  QT_FED  QT_MUN  QT_EST  QT_PRI  ESCOLA_FED  \
0        SP       34099      47   13853    6524   13675    0.137834   
1        MG       24293      82   13441    4294    6476    0.337546   
2        BA       19611      37   15175    1137    3262    0.188670   
3        MA       13635      31   10902    1313    1389    0.227356   
4        RJ       13229      68    6286    1593    5282    0.514022   
5        PA       12092      24    9970    1103     995    0.198478   
6        RS       11287      50    5650    2516    3071    0.442988   
7        CE       10648      35    7086     776    2751    0.328700   
8        PE       10219      26    5852    1085    3256    0.254428   
9        PR        9999      30    5459    2140    2370    0.300030   
10       SC        7376      39    4283    1378    1676    0.528742   
11       AM        6148      18    4941     789     400    0.292778   
12       PI        5849      23    4554     762     510    0.393230   
13    

In [12]:
federal = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)
colormap = linear.RdPu_09.scale(0, dfprincipal['ESCOLA_FED'].max())
geojson_path = 'BR_UF_2022/BR_UF_2022.json'
geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=dfprincipal,
    columns=['SIGLA_UF', 'ESCOLA_FED'],
    key_on='feature.properties.SIGLA_UF',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas federais',
    highlight=True,
).add_to(federal)

colormap.add_to(federal)
federal.add_child(colormap)
federal
federal.save('escolas_federais.html')

In [13]:
municipal = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)
colormap = linear.BuGn_09.scale(0, dfprincipal['ESCOLA_MUN'].max())
geojson_path = 'BR_UF_2022/BR_UF_2022.json'
geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=dfprincipal,
    columns=['SIGLA_UF', 'ESCOLA_MUN'],
    key_on='feature.properties.SIGLA_UF',
    fill_color='BuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas municipais',
    highlight=True,
).add_to(municipal)

colormap.add_to(municipal)
municipal.add_child(colormap)
municipal
municipal.save('escolas_municipais.html')

In [14]:
estadual = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)
colormap = linear.YlGn_09.scale(0, dfprincipal['ESCOLA_EST'].max())
geojson_path = 'BR_UF_2022/BR_UF_2022.json'
geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=dfprincipal,
    columns=['SIGLA_UF', 'ESCOLA_EST'],
    key_on='feature.properties.SIGLA_UF',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas estaduais',
    highlight=True,
).add_to(estadual)

colormap.add_to(estadual)
estadual.add_child(colormap)
estadual
estadual.save('escolas_estaduais.html')

In [15]:
privada = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)
colormap = linear.PuRd_09.scale(0, dfprincipal['ESCOLA_PRI'].max())
geojson_path = 'BR_UF_2022/BR_UF_2022.json'
geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=dfprincipal,
    columns=['SIGLA_UF', 'ESCOLA_PRI'],
    key_on='feature.properties.SIGLA_UF',
    fill_color='PuRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas privadas',
    highlight=True,
).add_to(privada)

colormap.add_to(privada)
privada.add_child(colormap)
privada
privada.save('escolas_privadas.html')